# Young Audiences Data

In [1]:
import pandas as pd
import glob
from collections import defaultdict
from fuzzywuzzy import process, fuzz
#import matplotlib.pyplot as plt

In [2]:
path = "/Users/jjgong/Desktop/ya-pay-equity/data/raw/"
# Use Glob to retrieve all of the file names
file_list = glob.glob(path+'*.xlsx')
# Sort files
file_list.sort()

In [3]:
file_list

['/Users/jjgong/Desktop/ya-pay-equity/data/raw/Performance Pair Pricing.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/Professional Learning Pricing.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/Single Assembly Pricing.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/Teaching Artist Demographics Collection 2021.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/Teaching Artist Demographics Collection Short 04 06 21.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/Three Year Sales Report.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/Workshop Pricing.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/artist_count_per_group.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/demographic_cleaned_part_1.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/demographic_cleaned_part_2.xlsx',
 '/Users/jjgong/Desktop/ya-pay-equity/data/raw/demographic_snapshot.xlsx']

## Data Upload

In [4]:
d = defaultdict(lambda: defaultdict())
for i,f in enumerate(file_list):
    d[i]['name'] = file_list[i][45:-5]
    d[i]['file'] = pd.read_excel(f)

In [5]:
# Get title and file
def get_name(d):
    print(d['name'])
    return d['file']

In [6]:
pd.set_option("max_columns", 500)

## Three Year Sales Data

In [7]:
three_year = get_name(d[5]).copy()
three_year.columns = [col.strip() for col in list(three_year.columns)]

Three Year Sales Report


In [8]:
# Mapping for IDs 
id_map = defaultdict()
for aid, an in zip(three_year['Artist Payroll ID'], three_year['Artist: Account Name']):
    if pd.isnull(aid)==False:
        id_map[an] = aid

In [9]:
# Map the unmapped artist IDs
three_year.loc[three_year[pd.isnull(three_year['Artist Payroll ID'])].index, ['Artist Payroll ID']] = three_year[pd.isnull(three_year['Artist Payroll ID'])]['Artist: Account Name'].apply(lambda x: d.get(x, float('NaN')))
# Literary Art to Literary Arts
three_year.loc[three_year[three_year['Art Form (General Discipline)']=='Literary Arts'].index, ['Art Form (General Discipline)']] = 'Literary Art'

In [11]:
#three_year.apply(lambda x: id_map.get(x['Artist: Account Name']) if pd.isnull(x['Artist Payroll ID']) else x, axis=1)

In [12]:
# Map missing names # Artist Payroll ID. Map some unmapped one
three_year['Artist Payroll ID'] = three_year.apply(lambda x: id_map.get(x['Artist: Account Name']) if pd.isnull(x['Artist Payroll ID']) else x, axis=1)['Artist Payroll ID']
#three_year['Artist Payroll ID'] = three_year.apply(lambda x: id_map.get(x['Artist: Account Name']) if pd.isnull(x['Artist Payroll ID']) else x, axis=1)['Artist: Account Name']

In [13]:
# Null Payroll IDs still
three_year[pd.isnull(three_year['Artist: Account Name'])]

,Artist Payroll ID,Artist: Account Name,Art Form (General Discipline),Contract Classification,Date,Contract #,Client Zip Code,Client,Billing Code,Component Type,Artist Fee,Sale Price,Artist Business name


In [14]:
three_year[pd.isnull(three_year['Artist Payroll ID'])]

,Artist Payroll ID,Artist: Account Name,Art Form (General Discipline),Contract Classification,Date,Contract #,Client Zip Code,Client,Billing Code,Component Type,Artist Fee,Sale Price,Artist Business name
2114,NaN,Deborah Owens,NaN,Admin/Project Dev,2021-03-15,21-0009889,08540,Young Audiences New Jersey,UWC YA Match,Teacher Stipend,200.0,200.0,Deborah Owens
2115,NaN,Derling Dance Arts,Dance,Professional Learning,2019-02-06,19-0008302,08540,Young Audiences New Jersey,Dodge Dance,Professional Learning,350.0,500.0,Derlling Dance Arts
2116,NaN,Dr. Ronah Harris,NaN,Admin/Project Dev,2020-10-02,21-0009574,08540,Young Audiences New Jersey,Virtual Learning Pilot,Artist Stipend,500.0,500.0,Dr. Ronah Harris
9540,NaN,Roxey Ballet,Dance,Professional Learning,2018-11-02,19-0008035,08540,Young Audiences New Jersey,Dodge Dance,Professional Learning,350.0,500.0,NaN


In [15]:
# Removes unwanted indexes that have a NULL Artist Payroll ID
three_year = three_year[~three_year.index.isin(list(three_year[pd.isnull(three_year['Artist Payroll ID'])].index))].reset_index(drop=True)

In [16]:
three_year[pd.isnull(three_year['Artist Payroll ID'])]

,Artist Payroll ID,Artist: Account Name,Art Form (General Discipline),Contract Classification,Date,Contract #,Client Zip Code,Client,Billing Code,Component Type,Artist Fee,Sale Price,Artist Business name


In [17]:
demographic = get_name(d[10])
dd = demographic.copy()
# Remove Unamed column
demographic = demographic.drop(labels=['Unnamed: 0'], axis=1)
# Drop duplicates
demographic = demographic.drop_duplicates(subset=['artist_id'], keep='last').reset_index(drop=True)

demographic_snapshot


In [18]:
# Retrieve top 10 components
top_components = list(three_year['Component Type'].value_counts()[:9].index)
top_components.remove('Travel')
top_components.remove('Materials')
top_components.append('Virtual Planning Meeting')

In [19]:
top_components

['Workshop',
 'Performance/Demo',
 'Virtual Workshop',
 'Professional Learning',
 'Virtual Performance',
 'Virtual Professional Learning',
 'Planning Meeting',
 'Virtual Planning Meeting']

In [20]:
# Group by
three_yr_grp = three_year.groupby(['Artist Payroll ID', 'Artist: Account Name', 'Art Form (General Discipline)', 'Contract Classification', 'Date', 'Contract #', 'Client Zip Code', 'Client', 'Billing Code', 'Component Type', 'Artist Business name']).sum()['Artist Fee'].reset_index().reset_index(drop=True)

In [21]:
# Retrieve only top performances. Remove other components
# # three_year_cnt = three_year[three_year['Component Type'].isin(top_components)].copy()

In [22]:
# Retrieve only top performances. Remove other components
three_yr_grp = three_yr_grp[three_yr_grp['Component Type'].isin(top_components)].copy()

In [23]:
len(three_yr_grp)

3745

In [24]:
three_yr_grp

,Artist Payroll ID,Artist: Account Name,Art Form (General Discipline),Contract Classification,Date,Contract #,Client Zip Code,Client,Billing Code,Component Type,Artist Business name,Artist Fee
0,0.0,10HL,Dance,Professional Learning,2018-11-15,19-0008036,08540,Young Audiences New Jersey,Dodge Dance,Professional Learning,10 Hairy Legs,350.00
2,0.0,Oyin Hardy,Dance,Residency,2018-10-04,19-0007894,08302,Cherry St Elem School,21st century,Workshop,Troupe Da-Da African Dance & Drum Ensemble,410.00
4,0.0,Oyin Hardy,Dance,Residency,2018-10-11,19-0007894,08302,Cherry St Elem School,21st century,Workshop,Troupe Da-Da African Dance & Drum Ensemble,410.00
6,0.0,Oyin Hardy,Dance,Residency,2018-10-18,19-0007894,08302,Cherry St Elem School,21st century,Workshop,Troupe Da-Da African Dance & Drum Ensemble,410.00
8,0.0,Oyin Hardy,Dance,Residency,2018-10-25,19-0007894,08302,Cherry St Elem School,21st century,Workshop,Troupe Da-Da African Dance & Drum Ensemble,410.00
...,...,...,...,...,...,...,...,...,...,...,...,...
6968,264.0,Summer Dawn,Theater,Workshop,2021-02-17,21-0009627,08610,George E. Wilson Elementary,Arts Lab- Hamilton,Virtual Workshop,Thinking In Full Color LLC,350.01
6969,264.0,Summer Dawn,Theater,Workshop,2021-02-23,21-0009627,08610,George E. Wilson Elementary,Arts Lab- Hamilton,Virtual Workshop,Thinking In Full Color LLC,350.01
6970,264.0,Summer Dawn,Theater,Workshop,2021-02-24,21-0009627,08610,George E. Wilson Elementary,Arts Lab- Hamilton,Virtual Workshop,Thinking In Full Color LLC,350.01
6971,264.0,Summer Dawn,Theater,Workshop,2021-03-01,21-0009627,08610,George E. Wilson Elementary,Arts Lab- Hamilton,Virtual Professional Learning,Thinking In Full Color LLC,350.00


### Review unique artists

In [25]:
# Retrieve unique artists
unique_artists = three_yr_grp.drop_duplicates(subset=['Artist Payroll ID', 'Artist: Account Name'])[['Artist Payroll ID', 'Artist: Account Name']].reset_index(drop=True)

In [26]:
joined = three_yr_grp.merge(demographic, how='outer', left_on='Artist Payroll ID', right_on='artist_id')

In [27]:
len(joined)

3749

In [28]:
# Remove certain artists
num_include = list(set(joined['Artist Payroll ID'])-set([95, 0, 20, 56]))

In [29]:
included_df = joined[joined['Artist Payroll ID'].isin(num_include)]

In [30]:
null_demo = included_df[pd.isnull(included_df['artist_id'])].copy().reset_index(drop=True)

In [31]:
demo = included_df[~pd.isnull(included_df['artist_id'])].copy().reset_index(drop=True)

In [32]:
len(null_demo)

924

In [33]:
len(demo)

2683

In [34]:
len(demo)/(len(null_demo)+len(demo))

0.743831438868866

In [35]:
demo['Artist Payroll ID'].nunique()

63

In [36]:
len(demo)/(len(null_demo)+len(demo))

0.743831438868866

In [37]:
null_demo['Artist Payroll ID'].nunique()

52

#### Insert Group Size

In [ ]:
d.keys()

In [58]:
grp_size = get_name(d[7]).copy()

artist_count_per_group


In [61]:
three_yr_grp[(three_yr_grp['Date']>=pd.Timestamp(2019,1,1)) & (three_yr_grp['Date']<pd.Timestamp(2020,1,1))]['Artist Payroll ID'].nunique()

108

In [62]:
demo[(demo['Date']>=pd.Timestamp(2019,1,1)) & (demo['Date']<pd.Timestamp(2020,1,1))]['Artist Payroll ID'].nunique()

60

In [59]:
grp_size = grp_size[pd.isnull(grp_size['status'])].copy()

In [60]:
grp_size['artist_id'].value_counts()

0      3
95     3
20     2
56     2
109    1
      ..
51     1
50     1
49     1
48     1
282    1
Name: artist_id, Length: 105, dtype: int64

In [ ]:
grp_size

In [63]:
join = demo[~pd.isnull(demo['Artist Payroll ID'])].copy()

In [64]:
join[(join['Date']>=pd.Timestamp(2019,1,1)) & (join['Date']<pd.Timestamp(2020,1,1))]['Artist Payroll ID'].nunique()

60

In [65]:
artists_remove = ['Anndee Hochman', 'Oyin Hardy']
artist_id_remove = [95, 20 , 56]

In [66]:
grp_size = grp_size[~grp_size['artist_account_name'].isin(artists_remove)]

In [67]:
grp_size = grp_size[~grp_size['artist_id'].isin(artist_id_remove)]

In [107]:
bla = join.merge(grp_size, how='left', on='artist_id')

In [116]:
[int(x) for x in list(bla[pd.isnull(bla['size'])]['Artist Payroll ID'].unique())]

[26, 40, 75, 76, 85, 91, 93, 98, 101, 104, 114, 115]

In [113]:
grp_size[grp_size['artist_id'].isin([26])]

,artist_id,artist_account_name,size,status


In [120]:
bla[pd.isnull(bla['size'])]['Artist: Account Name'].unique()

array(['Hua Hua Zhang', 'Gwendolyn Briley-Strand',
       'Mexico Beyond Mariachi', 'Li Liu', 'Soul Steps',
       'Almanac Dance Circus Theatre', 'Hip Hop Fundamentals',
       'Cello Fury', "Pat Cannon's Foot and Fiddle Dance Company",
       'The Seventh Principle', 'Queen Nur', 'Caryn Lin'], dtype=object)

In [90]:
df_mer = join.merge(grp_size, on='artist_id')

In [94]:
# df_mer[pd.isnull(df_mer['Artist Payroll ID'])]['artist_id']
# join[join['artist_id'].isin([0, 18, 19 , 21, 22, 28, 31, 33, 34, 35, 36, 37, 38, 49, 42, 43, 48, 49, 51, 53])]

In [96]:
cols = ['artist_account_name', 'artist_id',
       'Art Form (General Discipline)', 'Contract Classification', 'Date',
       'Contract #', 'Client Zip Code', 'Client', 'Billing Code',
       'Component Type', 'Artist Fee', 'respondent_id', 'age', 'gender', 
       'city', 'state', 'company', 'ethnicity', 'multi_ind', 'size']

In [97]:
df_mer = df_mer[cols].copy()

In [98]:
records = df_mer[(df_mer['Date']>=pd.Timestamp(2019,1,1)) & (df_mer['Date']<pd.Timestamp(2020,1,1))]

In [99]:
records.head()

,artist_account_name,artist_id,Art Form (General Discipline),Contract Classification,Date,Contract #,Client Zip Code,Client,Billing Code,Component Type,Artist Fee,respondent_id,age,gender,city,state,company,ethnicity,multi_ind,size
10,Rob Aptaker,16.0,Theater,Assembly,2019-01-11,19-0008149,07645,Memorial Elem School-Montvale,Program Services,Performance/Demo,292.0,1.220051e+10,55 to 64,male,Allentown,PA,NaN,caucasian,1.0,1.0
11,Rob Aptaker,16.0,Theater,Assembly,2019-01-18,19-0007901,07930,Bragg Elem School,Program Services,Performance/Demo,292.0,1.220051e+10,55 to 64,male,Allentown,PA,NaN,caucasian,1.0,1.0
12,Rob Aptaker,16.0,Theater,Assembly,2019-01-25,19-0008152,07649,Oradell Public School,Program Services,Performance/Demo,292.0,1.220051e+10,55 to 64,male,Allentown,PA,NaN,caucasian,1.0,1.0
13,Rob Aptaker,16.0,Theater,Assembly,2019-01-30,19-0007818,07661,Cherry Hill Elem School,Discount,Performance/Demo,278.0,1.220051e+10,55 to 64,male,Allentown,PA,NaN,caucasian,1.0,1.0
14,Rob Aptaker,16.0,Theater,Assembly,2019-01-30,19-0007942,07661,Roosevelt School,Discount,Performance/Demo,278.0,1.220051e+10,55 to 64,male,Allentown,PA,NaN,caucasian,1.0,1.0


In [83]:
res_19 = records.groupby(['ethnicity','size']).nunique()['artist_id'].reset_index()

In [105]:
res_19[res_19['size']==1]

,ethnicity,size,artist_id
0,african,1.0,4
2,asian,1.0,1
5,caucasian,1.0,22
11,latinx,1.0,5


In [100]:
res_19['artist_id']

0      4
1      1
2      1
3      1
4      1
5     22
6      4
7      1
8      4
9      1
10     1
11     5
12     2
Name: artist_id, dtype: int64

In [101]:
records['artist_id'].nunique()

48

In [ ]:
df_ind = df_mer[df_mer['size']==1].copy()

In [ ]:
new_cols = ['artist_account_name', 'artist_id', 'Art Form (General Discipline)',
       'Contract Classification', 'Date', 'Contract #', 'Client Zip Code',
       'Client', 'Billing Code', 'Component Type', 
       'respondent_id', 'age', 'gender', 'city', 'state', 
       'ethnicity', 'Artist Fee']

In [ ]:
df_ind = df_ind[new_cols].copy()

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
len(['artist_account_name', 'artist_id', 'Art Form (General Discipline)',
       'Contract Classification', 'Date', 'Contract #', 'Client Zip Code',
       'Client', 'Billing Code', 'Component Type', 'respondent_id', 'age',
       'gender', 'city', 'state', 'ethnicity'])

In [ ]:
x = np.array(df_ind.loc[:,['artist_id', 'Art Form (General Discipline)',
       'Contract Classification', 'Date', 'Contract #', 'Client Zip Code',
       'Client', 'Billing Code', 'Component Type', 'respondent_id', 'age',
       'gender', 'city', 'state', 'ethnicity']]).reshape(-1,15)

In [ ]:
y = np.array(df_ind['Artist Fee'])

In [ ]:
model = LinearRegression()

In [ ]:
pd.to_numeric(df_ind['Art Form (General Discipline)'], errors='coerce')

In [ ]:
df_ind['Art Form (General Discipline)'].apply(pd.to_numeric)

In [ ]:
df_ind.loc[:,['artist_id', 'Art Form (General Discipline)',
       'Contract Classification', 'Date', 'Contract #', 'Client Zip Code',
       'Client', 'Billing Code', 'Component Type', 'respondent_id', 'age',
       'gender', 'city', 'state', 'ethnicity']].apply(pd.to_numeric, errors='coerce')

In [ ]:
x.apply(pd.to_numeric, errors='coerce')

In [ ]:
model.fit(x, y)

In [ ]:
#join[join['artist_id']==56]

In [ ]:
p = three_year[pd.isnull(three_year['Artist Payroll ID'])]
p = three_year.drop_duplicates(['Artist Payroll ID', 'Artist: Account Name'])

In [ ]:
demo[demo['age']=='75 or older']

In [ ]:
demo.to_excel('demographics.xlsx')